In [1]:
%load_ext jupyter_black

In [2]:
import numpy as np
import scipy as sp
from scipy.linalg import svd, qr

In [3]:
np.set_printoptions(precision=4, suppress=True)

In [4]:
A = np.array([[3.0, 2.0, -1.0], [2.0, -4, 5.0], [1.0, 1.0, 1.0], [2.0, 2.0, 2.0]])
b = np.array([1.0, 2.0, 3.0, 6.0])
A

array([[ 3.,  2., -1.],
       [ 2., -4.,  5.],
       [ 1.,  1.,  1.],
       [ 2.,  2.,  2.]])

In [5]:
b

array([1., 2., 3., 6.])

In [6]:
sol = np.linalg.lstsq(A, b)[0]
sol

array([-0.1111,  1.4815,  1.6296])

## Task 13: gelss
Implement `gelss` (with numpy, python and scipy.svd).

In [7]:
def gelss(A: np.ndarray, b: np.array) -> np.array:
    """
    Computes svd decomposition of the weights matrix and solves least squares problem by amtrix multiplication.

    Parameters:
    A (ndarray): Matrix of weights (n x m)
    b   (array): Biases vector (m x 1)

    Returns:
    x (array): least squares problem solution
    """
    
    U, D, Vh = svd(A, full_matrices=False)

    for i in range(D.shape[0]):
        D[i] = 1.0 / D[i] if D[i] > 1e-9 else 0.0

    D = D * np.eye(D.shape[0])

    return (Vh.T @ D @ U.T @ b.reshape(-1, 1)).flatten()

## Task 14: gelsd
Implement `gelsd` (with numpy, python and scipy.svd).

First, we have to implement a bidiagonalization algorithm. Here I used [the Golub-Kahan Bidiagonalization algorithm](https://www.active-analytics.com/blog/householder-bigiag-in-d/).

In [8]:
def bidiag(A: np.ndarray) -> (np.ndarray, np.ndarray, np.ndarray):
    """
    An implementation of Golub-Kahan Bidiagonalization algorithm.

    Parameters:
    A (ndarray): n x m Matrix to be decomposed

    Returns:
    U (ndarray): Left Householder matrix (n x n)
    B (ndarray): Bidiagonal matrix (n x m)
    V (ndarray): Right Householder matrix (m x m)
    """

    def sign(t):
        return 1.0 if t >= 0.0 else -1.0

    def house_mtx(vect: np.array, ndim: int = None) -> np.ndarray:
        """
        Transofrms a Householder vector into corresponding Householder matrix(operator)

        Parameters:
        vect (ndarray): Householder vector in question.
        ndim     (int): Dimension to expand the operator to.

        Returns:
        P (ndarray): Householder operator
        """
        r = vect.shape[0]

        if ndim is None:
            ndim = vect.shape[0]

        vect = vect.reshape(-1, 1)
        P = np.eye(ndim)
        P[(ndim - r) :, (ndim - r) :] = P[(ndim - r) :, (ndim - r) :] - 2.0 * (
            vect @ vect.T
        )

        return P

    n = A.shape[0]
    m = A.shape[1]
    B = A.copy()
    U = np.eye(n)
    V = np.eye(m)

    for k in range(m):

        x = B[k:, k].flatten()
        u = x.copy()
        u[0] += sign(x[0]) * np.linalg.norm(x)
        u = u / np.linalg.norm(u)

        transform = house_mtx(u, n)
        U = transform @ U
        B[k:, k:] = house_mtx(u) @ B[k:, k:]

        if k < (m - 2):

            x = B[k, k + 1 :].flatten()
            v = x.copy()
            v[0] += sign(x[0]) * np.linalg.norm(x)
            v = v / np.linalg.norm(v)

            transform = house_mtx(v, m)
            V = transform @ V
            B[k:, (k + 1) :] = B[k:, (k + 1) :] @ house_mtx(v)

    return (U, B, V)

Let's see if bidiagonalization algorithm works

In [9]:
bidiag(A)

(array([[-0.7071, -0.4714, -0.2357, -0.4714],
        [ 0.6771, -0.6663, -0.1397, -0.2794],
        [ 0.2039,  0.5778, -0.3534, -0.7069],
        [-0.    ,  0.    , -0.8944,  0.4472]]),
 array([[-4.2426,  2.9155, -0.    ],
        [-0.    ,  3.7613, -4.3634],
        [-0.    , -0.    ,  3.7834],
        [-0.    , -0.    , -0.    ]]),
 array([[ 1.    ,  0.    ,  0.    ],
        [ 0.    , -0.2425, -0.9701],
        [ 0.    , -0.9701,  0.2425]]))

In [10]:
us, bs, vs = bidiag(A)

if (np.abs(us.T @ bs @ vs.T - A) < 1e-9).all():
    print(f"This is indeed correct Golub-Kahan decomposition!")
else:
    print((us.T @ bs @ vs.T - A))
    print(
        f"Something went wrong, maximal deviation is {np.max(np.abs(us.T @ bs @ vs.T - A))} at index {(int(np.argmax(np.abs(us.T @ bs @ vs.T - A)) // bs.shape[1])+1, int(np.argmax(np.abs(us.T @ bs @ vs.T - A)) % bs.shape[1])+1)}"
    )

This is indeed correct Golub-Kahan decomposition!


In [11]:
def bls(A: np.ndarray, b: np.array) -> np.array:
    """
    Computes the solution of the bidiagonal least squares problem.

    Parameters:
    A (ndarray): Upper bidiagonal matrix (n x m)
    b   (array): Biases vector (m x 1)

    Returns:
    x (array): least squares problem solution
    """

    n = A.shape[0]
    m = A.shape[1]
    x = np.zeros((m,))

    if m <= n:
        x[-1] = b[m - 1] / A[m - 1, m - 1]

        for i in range(1, m):
            x[-1 - i] = (b[m - i - 1] - x[-i] * A[m - i - 1, m - i]) / A[
                m - i - 1, m - i - 1
            ]
    else:
        x[m - 1] = b[-1] / A[m - 1, m - 1]

        for i in range(1, m):
            x[m - 1 - i] = (b[-1 - i] - x[m - i] * A[m - i - 1, m - i]) / A[
                m - i - 1, m - i - 1
            ]

    return x

In [12]:
def gelsd(A: np.ndarray, b: np.array) -> np.array:
    """
    Computes biadiagonal decomposition of the weights matrix and solves BLS problem using LBD algorithm, then applies the householder transformations to the result.

    Parameters:
    A (ndarray): Matrix of weights (n x m)
    b   (array): Biases vector (m x 1)

    Returns:
    x (array): least squares problem solution
    """

    U, B, Vh = bidiag(A)

    beta = (U @ b.reshape(-1, 1)).flatten()

    y = bls(B, beta)

    return Vh @ y

## Task 15: gelsy
Implement `gelsy` (with numpy, python and scipy.qr).

In [13]:
def gelsy(A, b, rcond=None):
    """
    Solves the least squares problem Ax = b using QR decomposition with column pivoting.

    Parameters:
    A (ndarray): Matrix of weights (n x m)
    b   (array): Biases vector (m x 1)

    Returns:
    x   (array): least squares problem solution
    """
    Q, R, P = qr(A, pivoting=True)
    
    rank = np.sum(np.abs(np.diag(R)) > 1e-9)

    R11 = R[:rank, :rank]
    Qt_b = Q.T @ b

    y = np.zeros(R.shape[1])
    y[:rank] = np.linalg.solve(R11, Qt_b[:rank])

    x = np.zeros(R.shape[1])
    x[P] = y

    return x

In [14]:
print(f"numpy.linalg.lstsq solution:\n    {sol}")

print(f"Checking:\n    Ax = {np.dot(A, sol)}\n    b = {b}")

numpy.linalg.lstsq solution:
    [-0.1111  1.4815  1.6296]
Checking:
    Ax = [1. 2. 3. 6.]
    b = [1. 2. 3. 6.]


In [15]:
x = gelss(A, b)
print(f"gelss solution x:\n    {x}")

print(f"Checking:\n    Ax = {np.dot(A, x)}\n    b = {b}")

gelss solution x:
    [-0.1111  1.4815  1.6296]
Checking:
    Ax = [1. 2. 3. 6.]
    b = [1. 2. 3. 6.]


In [16]:
x = gelsd(A, b)
print(f"gelsd solution x:\n    {x}")

print(f"Checking:\n    Ax = {np.dot(A, x)}\n    b = {b}")

gelsd solution x:
    [-0.1111  1.4815  1.6296]
Checking:
    Ax = [1. 2. 3. 6.]
    b = [1. 2. 3. 6.]


In [17]:
x = gelsy(A, b)
print(f"gelsy solution x:\n    {x}")

print(f"Checking:\n    Ax = {np.dot(A, x)}\n    b = {b}")

gelsy solution x:
    [-0.1111  1.4815  1.6296]
Checking:
    Ax = [1. 2. 3. 6.]
    b = [1. 2. 3. 6.]


## Task 16: Cholesky

Fix bugs in the Cholesky decomposition algorithm.

In [18]:
def cholesky(A: np.ndarray) -> np.ndarray:
    """
    Performs a Cholesky decomposition of the matrix A.

    Parameters:
    A (ndarray): Positive definite matrix to be decomposed (n x n)

    Returns:
    L (ndarray): Lower-triangular matrix such that A := L @ L*
    """

    L = np.zeros_like(A, dtype=np.float64)
    n = A.shape[0]

    for i in range(n):
        L[i, i] = np.sqrt(A[i, i] - np.dot(L[i, :i], L[i, :i]))
        L[i + 1 :, i] = (A[i + 1 :, i] - np.dot(L[i + 1 :, :i], L[i, :i])) / L[i, i]
    return L

Let's define a positive definite symmetric matrix and test our algorithm on it.

In [19]:
mtx = np.array([[1e3, 1.0], [1.0, 1e-2]])
mtx

array([[1000.  ,    1.  ],
       [   1.  ,    0.01]])

In [21]:
L = cholesky(mtx)

if ((L @ L.T - mtx) < 1e-9).all():
    print(f"L = \n{L}\n\nThis is indeed Cholesky decomposition!")
else:
    print(f"Something went wrong...")

L = 
[[31.6228  0.    ]
 [ 0.0316  0.0949]]

This is indeed Cholesky decomposition!
